Transformer From Scratch 
========================

Reading along Dan Jurafsky and James H. Martin's [Speech and Language Processing](https://web.stanford.edu/~jurafsky/slp3/) book, I decided to follow through Chapter 8 of their book to implement a Transformer using Pytorch. It is my goal to have a working transformer which I can use to train on the guitar dataset. I know some linear algebra, the book essentially gives the entire algorithm in terms of linear algebra, and pytorch provides a nice but still very informative abstractions for doing linear algebra. I had no reason not to pursue this project on top of whatever I proposed to do initially. 

## 

Attention Layer
---------------

At the heart of Transformer is the **attention layer**. It is a mechanism that allows words(tokens) to gain contextual meaning from their surrounding words(tokens). It can have multiple **"heads"**, where each "head" can be thought of as a specialist who asks particular set of questions given some data. For instance, one head could focus solely on grammar while another could instead focus on sentiments (even though that might not be exactly what occurs under the hood).

Each head's job, then, is to ask the right kinds of *questions* to choose which of previous words it has seen matters the most to the current word. To do this, each head consists of three main components: **Query**, **Key**, and **Value** weight matrices. 

<!-- 
    essentially, what it is at the end of the day is weighted sum, but it's obviously lot more complicated than that
    don't forget to write out the equations that I have referenced
    maybe throw in some pictures
    say something about how masking and softmax is used to determine what key's to focus on
    also explain how results from different heads are consolidated at the end
--!>

In [1]:
from myTransformer import *
batch_size = 10
N = 10
model_dim = 24
num_heads = 4
key_dim = 3

M = 8
X = torch.rand((batch_size, N, model_dim)) # batch_size is 10, 3 words represented as dim (1, 4) tensors
Y = torch.rand((batch_size, M, model_dim)) # 3 words represented as dim (1, 4) tensors
mask = torch.tensor([[0 if i>= j else -torch.inf for j in range(N)] for i in range(N)])

multihead_attention = AttentionLayer(model_dim=model_dim, key_dim=key_dim, num_heads=num_heads)
multihead_attention(X, H_enc=Y, mask=mask).shape
#multihead_attention.to("cuda")
#multihead_attention(X.to("cuda"), Y.to("cuda"), mask=mask.to("cuda"))

torch.Size([10, 10, 24])

In [2]:
encoder_block = TransformerBlock(N=N, model_dim=model_dim, key_dim=key_dim, hidden_dim=8, num_heads=num_heads)
decoder_block = TransformerBlock(N=N, model_dim=model_dim, key_dim=key_dim, hidden_dim=8, num_heads=num_heads, cross_attention=True)
encoder_block(X,mask=mask)
decoder_block(X, Y)

tensor([[[ 0.4755,  1.3892,  0.9689,  ...,  0.1691,  0.0524,  0.6801],
         [ 0.9499,  1.1286, -0.0386,  ..., -0.4186,  0.2665,  0.9138],
         [ 0.2616,  0.5613,  0.8213,  ..., -0.5264,  0.1686,  1.1733],
         ...,
         [ 0.9664,  0.0000,  0.0000,  ...,  0.0327, -0.3498,  0.0000],
         [ 1.4042,  0.9370,  1.2821,  ..., -0.3033, -0.0830,  0.0000],
         [ 0.6124,  0.7261,  0.6314,  ...,  0.1379, -0.0000,  0.0000]],

        [[ 0.9795,  0.8361,  0.6765,  ..., -0.1547,  0.6105,  0.0418],
         [ 0.9273,  1.3577,  0.5522,  ...,  0.1998,  0.0080,  1.2078],
         [ 0.3862,  0.5071,  1.2673,  ...,  0.3771,  0.5118,  0.3256],
         ...,
         [ 0.3525,  0.0000,  1.1488,  ...,  0.1131, -0.0866,  1.1911],
         [ 0.6050,  0.4014,  1.2233,  ..., -0.0284, -0.0000,  0.1443],
         [ 0.5638,  0.9558,  0.7586,  ..., -0.7112, -0.1673,  1.2769]],

        [[ 1.1975,  1.0133,  0.2820,  ..., -0.0092,  0.4525,  1.0359],
         [ 1.3390,  0.0000,  0.4201,  ..., -0

In [3]:
stack = TransformerStack(N=N, model_dim=model_dim, key_dim=key_dim, hidden_dim=8, num_heads=num_heads, num_stack=9)
stack.state_dict()
stack.train()
stack(X)

tensor([[[-0.1946, -1.1348,  1.1489,  ...,  6.2648,  0.1012, -0.4266],
         [-1.7375, -1.5236, -0.1893,  ...,  4.9815,  3.4274, -0.2885],
         [-0.8263, -0.2099,  0.8725,  ...,  0.5749,  2.6182,  1.0176],
         ...,
         [-0.4358, -1.1794,  1.4692,  ...,  5.1027,  0.6614,  0.8013],
         [-0.0000, -0.0000, -0.1040,  ...,  3.3305, -0.7707,  0.7355],
         [-0.9725, -1.9939,  0.9844,  ...,  0.0000, -0.1960, -0.0283]],

        [[ 2.2090,  1.0609,  0.9493,  ...,  4.4807,  1.7722,  1.2402],
         [-1.9897,  0.9965,  0.0000,  ...,  1.6594, -0.2985, -0.0936],
         [-0.1578, -0.8394,  0.8542,  ...,  3.7079, -0.3406,  0.6509],
         ...,
         [ 0.6368, -2.0955, -1.4161,  ..., -0.0328,  0.4694,  1.1917],
         [-0.4128,  0.9801,  1.2094,  ...,  0.4574,  0.4304, -0.5428],
         [ 2.2097, -0.2051, -0.3172,  ...,  1.5958, -0.9028, -0.0941]],

        [[ 0.0947, -0.8518,  0.8360,  ...,  5.0722,  1.0791, -0.0000],
         [ 0.1222,  1.6562,  1.3867,  ...,  2

In [4]:
# from https://pytorch-tutorials-preview.netlify.app/beginner/transformer_tutorial.html
# i don't completely understand positional encoding yet, but I have built the intuition that 
# it is analogous to how binary numbers encode numbers; smaller bits flips more frequently 
# than larger bits; this is modeled by the sinusodial waves 
# it also takes advantage of linearity of trigonometric addition formulas, which supposedly 
# helps the model to figure out relative positioning...
# https://medium.com/thedeephub/positional-encoding-explained-a-deep-dive-into-transformer-pe-65cfe8cfe10b 
class PositionalEncoding(nn.Module):

    def __init__(self, model_dim: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, model_dim, 2) * (-math.log(10000.0) / model_dim))
        pe = torch.zeros(max_len, 1, model_dim)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, X):
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        X = X + self.pe[:X.size(0)]
        return self.dropout(X)

In [5]:
# what I need to do to finish up the encoder_decoder architecture 
# the only difference for the decoder architecture is the cross attention layer `
# which is much like the self-attension layer except that it is using both the final 
# H of the encoder and that of decoder to do query-key matching, thus decoder needs to 
# take in memory from encoder
pos = PositionalEncoding(model_dim=model_dim)


In [6]:
import dac
from ICMTSMTGuitarData import *
from jupyter_audio_utils import *

model_path = dac.utils.download(model_type="44khz")
model = dac.DAC.load(model_path, weights_only=True).eval()

mono_data2 = ICMTSMTGuitarDataMono(DEFAULT_PEDAL_PROBS)
test_2 = mono_data2[random.randint(0, len(mono_data2)-1)] 

waveform, sr = test_2[0] 
play_audio(*test_2[0])

x = model.preprocess(waveform, sr)

with torch.no_grad():
    z, codes, latents, _, _ = model.encode(x.unsqueeze(dim=0))

audio_tokens = z.transpose(-2,-1) #(batch_size, seq_len, model_dim)
N = audio_tokens.shape[-2] 
model_dim = audio_tokens.shape[-1] 
key_dim = 256
hidden_dim = 512
num_heads=4

testing = TransformerStack(N=N, model_dim=model_dim, key_dim=key_dim, hidden_dim=256, num_heads=num_heads, num_stack=1)
testing(audio_tokens)

2026-01-29 14:30:40.456912: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-29 14:30:40.531145: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-29 14:30:40.546895: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-01-29 14:30:40.810181: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

tensor([[[ 5.7094, -8.7209,  1.0299,  ...,  2.6714,  1.0670,  0.9740],
         [ 3.1440, -6.9980,  1.9141,  ...,  0.0000,  3.8013,  0.0000],
         [ 3.2578, -5.8899,  1.5415,  ...,  5.5905,  3.3019,  2.7232],
         ...,
         [-1.4050, -4.1484,  2.6340,  ..., -0.7271, -1.0902,  0.5627],
         [ 0.0000, -4.4983,  1.2675,  ...,  0.0000,  7.8973,  5.5228],
         [ 6.4985, -5.4442,  1.1464,  ..., -1.4488,  0.2719, -3.6595]]],
       grad_fn=<MulBackward0>)

In [7]:
from myTransformer import *

# blueprint
class EncoderDecoder(nn.Module): 
    def __init__(
        self, 
        N, 
        model_dim,
        key_dim,
        encoder_mask,
        encoder_hidden_dim, 
        encoder_num_stack,
        encoder_num_heads,
        decoder_hidden_dim, 
        decoder_num_stack,
        decoder_num_heads,
        decoder_vocab
    ):
        pass
        self.positional_encoder = PositionalEncoding()
        self.encoder_stack = TransformerStack(
            N=N, 
            model_dim=model_dim, 
            key_dim=key_dim, 
            hidden_dim=encoder_hidden_dim, 
            num_heads=encoder_num_heads, 
            num_stack=encoder_num_stack
        )
        self.decoder_stack = TransformerStack(
            N=N, 
            model_dim=model_dim, 
            key_dim=key_dim, 
            hidden_dim=decoder_hidden_dim, 
            num_heads=decoder_num_heads, 
            num_stack=decoder_num_stack,
            cross_attention=True
        )
        self.language_head = None
        self.decoder_vocab = decoder_vocab
        self.mask = encoder_mask # do the self register thingy
    def forward(self, X): # (batch_size, N, model_dim)
        X = self.positional_encoder(X)
        H1 = self.encoder_stack(X, mask=self.mask)
        H2 = self.decoder_stack(X, H1)
        Y = self.language_head(H2,...) # not implemented yet

        return Y